In [1]:
import os
import pandas as pd
from datetime import datetime 
from se4g_dir import set_the_working_directory
# Set the working directory
set_the_working_directory()

Current directory:  c:\Users\Utente\Documents\GitHub\SE4GEO-Lab\db_helper
Current working directory: c:\Users\Utente\Documents\GitHub\SE4GEO-Lab


Create se4g_pollution_main.csv

In [3]:
from se4g_helper import build_dataframe, update_dataset

df_columns=['station_code', 'station_name', 'station_altitude', 'network_countrycode', 'pollutant', 'value_datetime_begin',
            'value_datetime_end', 'value_datetime_updated', 'value_numeric', 'samplingpoint_x', 'samplingpoint_y'
            ]
data_type = ['VARCHAR', 'VARCHAR', 'FLOAT', 'CHAR(2)', 'VARCHAR', 'VARCHAR',
             'VARCHAR', 'VARCHAR', 'FLOAT', 'FLOAT', 'FLOAT'
             ]

In [4]:
fileName = "se4g_pollution_main.csv" # execution time: 1m23s -> 1m51s
folder_out = 'data_prova'


columns = df_columns

data_type = data_type


for item_name in os.listdir(folder_out):
    item_path = os.path.join(folder_out, item_name)

    if os.path.isdir(item_path):
        new_df = build_dataframe(item_name, folder_out = folder_out )

        update_dataset(new_df, folder_out = folder_out, fileName = fileName)

AD_SO2.csv
<_io.TextIOWrapper name='data_prova\\2023-05-22_18_04_21\\AD_SO2.csv' mode='r' encoding='utf-8-sig'>
AD_NO.csv
AD_NO2.csv
AD_CO.csv
<_io.TextIOWrapper name='data_prova\\2023-05-22_18_04_21\\AD_CO.csv' mode='r' encoding='utf-8-sig'>
AD_PM10.csv
<_io.TextIOWrapper name='data_prova\\2023-05-22_18_04_21\\AD_PM10.csv' mode='r' encoding='utf-8-sig'>
AT_SO2.csv
<_io.TextIOWrapper name='data_prova\\2023-05-22_18_04_21\\AT_SO2.csv' mode='r' encoding='utf-8-sig'>
AT_NO.csv
AT_NO2.csv
AT_CO.csv
<_io.TextIOWrapper name='data_prova\\2023-05-22_18_04_21\\AT_CO.csv' mode='r' encoding='utf-8-sig'>
AT_PM10.csv
<_io.TextIOWrapper name='data_prova\\2023-05-22_18_04_21\\AT_PM10.csv' mode='r' encoding='utf-8-sig'>
BA_SO2.csv
<_io.TextIOWrapper name='data_prova\\2023-05-22_18_04_21\\BA_SO2.csv' mode='r' encoding='utf-8-sig'>
BA_NO.csv
BA_NO2.csv
BA_CO.csv
<_io.TextIOWrapper name='data_prova\\2023-05-22_18_04_21\\BA_CO.csv' mode='r' encoding='utf-8-sig'>
BA_PM10.csv
<_io.TextIOWrapper name='data_p

Create se4g_pollution_dashboard.csv

In [2]:
source_df = "se4g_pollution_main.csv" # execution time: 1m23s -> 1m51s
folder_out = 'data_prova'
full_path = os.path.join(folder_out, source_df)

columns = ['pollutant', 'country', 'month_day', 'value_numeric_mean']

country = {'AD': 'Andorra', 'SE': 'Sweden', 'DE': 'Germany', 'CY': 'Undefined', 'BE': 'Belgium',
           'FI': 'Finland', 'ES': 'Spain', 'CZ': 'Czech Republic', 'BG': 'Bulgaria', 'BA': 'Bosnia and Herzegovina',
           'EE': 'Estonia', 'CH': 'Switzerland', 'AT': 'Austria', 'DK': 'Denmark'}
   
if os.path.isfile(full_path):
    dashboard_df = pd.read_csv(full_path)
    # Convert 'value_datetime_end' to datetime objects and extract the day
    datetime_objects = dashboard_df['value_datetime_end'].apply(lambda x: datetime.strptime(x, '%Y-%m-%d %H:%M:%S%z'))
    dashboard_df['month_day'] = datetime_objects.apply(lambda x: x.strftime('%m%d'))

    # Compute daily mean of 'value_numeric' for each 'pollutant' and 'network_countrycode'
    daily_mean = dashboard_df.groupby(['pollutant', 'network_countrycode', 'month_day'])['value_numeric'].mean().reset_index()
    
    # Merge the daily mean back to the original dataframe
    dashboard_df = dashboard_df.merge(daily_mean, on=['pollutant', 'network_countrycode', 'month_day'], suffixes=('', '_mean'))    

    dashboard_df['country'] = dashboard_df['network_countrycode'].map(country)
    dashboard_df = dashboard_df[columns].copy()
    dashboard_df = dashboard_df[dashboard_df['pollutant'] != 'O3']
    dashboard_df = dashboard_df.drop_duplicates().reset_index(drop=True)
    dashboard_df = dashboard_df.sort_values('month_day')

    dashboard_df.to_csv('data_prova/se4g_dashboard_dataset.csv', index=False)
    print("Dataset ",full_path," created")
    
else: 
        print("Dataset ",full_path," does not exist")
        print(os.getcwd())

Dataset  data_prova\se4g_pollution_main.csv  created


Check empty csv file

In [ ]:
file_path = 'data_prova/31-05-2023_09_15_34/ES_PM10.csv'
file_path = 'data_prova/31-05-2023_10_53_20/ES_PM10.csv'
print(file_path)

with open(file_path, 'r') as file:
    print(file)
    first_line = file.readline().strip()

if not first_line.startswith('<!DOCTYPE html'): 
    #print(fileName,'exist')

    df_temp = pd.read_csv(file_path)
    print(df_temp['network_countrycode'].head())

Download reques

In [ ]:
import requests
def download_request(COUNTRIES= 'ES',
                        POLLUTANTS= 'PM10',
                        folder_out = 'data_prova'):
    print ('-----------------------------------------------------------------------')
    # Set download url
    # https://discomap.eea.europa.eu/map/fme/AirQualityUTDExport.htm
    ServiceUrl = "http://discomap.eea.europa.eu/map/fme/latest"


    dir = datetime.now().strftime("%d-%m-%Y_%H_%M_%S")

    if not os.path.exists(os.path.join(folder_out, dir)):
        if not os.path.exists(folder_out):
            os.mkdir(folder_out)
        os.mkdir(os.path.join(folder_out, dir))
        print(dir,'directory created')
        
    for country in COUNTRIES:
        for pollutant in POLLUTANTS:
            fileName = "%s_%s.csv" % (country, pollutant)
            downloadFile = '%s/%s_%s.csv' % (ServiceUrl, country, pollutant)
            #Download and save to local path
            print('Downloading: %s' % downloadFile )

            file = requests.get(downloadFile).content
            full_file = os.path.join(folder_out, dir, fileName)
			


            '''with codecs.open(full_file, 'r', encoding='utf-8-sig') as file:
                content = file.read()'''

            # Modify the content as needed
            with codecs.open(full_file, 'w', encoding='utf-8') as output:
                output.write(file)



            '''output = open(full_file, 'wb')
            output.write(file)
            output.close()'''
            print ('Saved locally as: %s ' % full_file)
            print ('-----')
    print ('Download finished')
    return dir

dirr = download_request()

In [ ]:
def download_request(COUNTRIES=['ES'], POLLUTANTS=['PM10'], folder_out='data_prova'):
    print('-----------------------------------------------------------------------')
    # Set download url
    # https://discomap.eea.europa.eu/map/fme/AirQualityUTDExport.htm
    ServiceUrl = "http://discomap.eea.europa.eu/map/fme/latest"

    dir = datetime.now().strftime("%d-%m-%Y_%H_%M_%S")

    if not os.path.exists(os.path.join(folder_out, dir)):
        if not os.path.exists(folder_out):
            os.mkdir(folder_out)
        os.mkdir(os.path.join(folder_out, dir))
        print(dir, 'directory created')

    for country in COUNTRIES:
        for pollutant in POLLUTANTS:
            fileName = "%s_%s.csv" % (country, pollutant)
            downloadFile = '%s/%s_%s.csv' % (ServiceUrl, country, pollutant)
            # Download and save to local path
            print('Downloading: %s' % downloadFile)

            file_content = requests.get(downloadFile).content
            file_content_str = file_content.decode('utf-8-sig')

            full_file = os.path.join(folder_out, dir, fileName)

            with open(full_file, 'w', encoding='utf-8') as output:
                output.write(file_content_str)

            print('Saved locally as: %s ' % full_file)
            print('-----')
    print('Download finished')
    return dir

dirr = download_request()

In [ ]:
def download_request(COUNTRIES=['ES'], POLLUTANTS=['PM10'], folder_out='data_prova'):
    print('-----------------------------------------------------------------------')
    # Set download url
    # https://discomap.eea.europa.eu/map/fme/AirQualityUTDExport.htm
    ServiceUrl = "http://discomap.eea.europa.eu/map/fme/latest"

    dir = datetime.now().strftime("%d-%m-%Y_%H_%M_%S")

    if not os.path.exists(os.path.join(folder_out, dir)):
        if not os.path.exists(folder_out):
            os.mkdir(folder_out)
        os.mkdir(os.path.join(folder_out, dir))
        print(dir, 'directory created')

    for country in COUNTRIES:
        for pollutant in POLLUTANTS:
            fileName = "%s_%s.csv" % (country, pollutant)
            downloadFile = '%s/%s_%s.csv' % (ServiceUrl, country, pollutant)
            # Download and save to local path
            print('Downloading: %s' % downloadFile)

            file_content = requests.get(downloadFile).content
            file_content_str = file_content.decode('utf-8-sig')

            full_file = os.path.join(folder_out, dir, fileName)

            with codecs.open(full_file, 'w', encoding='utf-8') as output:
                output.write(file_content_str)

            print('Saved locally as: %s ' % full_file)
            print('-----')
    print('Download finished')
    return dir

dirr = download_request()

Rename Folders

In [7]:
import os
from datetime import datetime

# Get the current datetime in the format 'YYYY-MM-DD_HH_MM_SS'
current_datetime = datetime.now().strftime("%Y-%m-%d_%H_%M_%S")

# Directory path where the folders are located
directory = 'data'

# Iterate over the folders in the directory
for folder_name in os.listdir(directory):
    print(folder_name)
    if folder_name.startswith(datetime.now().strftime("%d-%m-%Y_%H_%M_%S")):
        # Create the new folder name by replacing the format 'DD-MM-YYYY_HH_MM_SS' with 'YYYY-MM-DD_HH_MM_SS'
        new_folder_name = folder_name.replace(current_datetime, current_datetime[:10])

        # Full paths of the old and new folders
        old_folder_path = os.path.join(directory, folder_name)
        new_folder_path = os.path.join(directory, new_folder_name)

        # Rename the folder
        os.rename(old_folder_path, new_folder_path)

06-06-2023_09_05_09
07-06-2023_14_09_19
22-05-2023_18_04_21
admins.csv
registrations.csv
se4g_dashboard_dataset.csv
se4g_pollution_dataset.csv


In [9]:
import os
from datetime import datetime

# Directory path where the folders are located
directory = 'data_prova'

# Iterate over the folders in the directory
for folder_name in os.listdir(directory):
    print(folder_name)
    if len(folder_name) == 19 and folder_name[2] == '-' and folder_name[5] == '-':
        # Extract the date and time parts from the folder name
        folder_date = folder_name[:10]
        folder_time = folder_name[11:]

        # Create the new folder name by rearranging the date parts to 'YYYY-MM-DD_HH_MM_SS'
        new_folder_name = folder_date[6:] + '-' + folder_date[3:5] + '-' + folder_date[:2] + '_' + folder_time

        # Full paths of the old and new folders
        old_folder_path = os.path.join(directory, folder_name)
        new_folder_path = os.path.join(directory, new_folder_name)

        # Rename the folder
        os.rename(old_folder_path, new_folder_path)


01-06-2023_10_39_44
01-06-2023_17_04_17
02-06-2023_09_02_53
04-06-2023_20_37_01
05-06-2023_11_03_18
05-06-2023_16_42_44
06-06-2023_09_05_09
07-06-2023_14_09_19
22-05-2023_18_04_21
22-05-2023_21_08_44
24-05-2023_09_00_18
24-05-2023_13_19_14
25-05-2023_12_16_05
26-05-2023_11_53_38
26-05-2023_15_48_03
29-05-2023_09_22_24
29-05-2023_23_22_53
30-05-2023_09_13_17
30-05-2023_21_48_50
31-05-2023_09_15_34
31-05-2023_10_53_20
se4g_dashboard_dataset.csv
se4g_pollution_dataset.csv
se4g_pollution_main.csv
